# **Homework**: Data talks club data engineering zoomcamp Data loading workshop

Hello folks, let's practice what we learned - Loading data with the best practices of data engineering.

Here are the exercises we will do




# 1. Use a generator

Remember the concept of generator? Let's practice using them to futher our understanding of how they work.

Let's define a generator and then run it as practice.

**Answer the following questions:**

- **Question 1: What is the sum of the outputs of the generator for limit = 5?**
- **Question 2: What is the 13th number yielded**

I suggest practicing these questions without GPT as the purpose is to further your learning.

In [1]:
def square_root_generator(limit):
    n = 1
    while n <= limit:
        yield n ** 0.5
        n += 1

# Example usage:
limit = 5
generator = square_root_generator(limit)

for sqrt_value in generator:
    print(sqrt_value)


1.0
1.4142135623730951
1.7320508075688772
2.0
2.23606797749979


# 2. Append a generator to a table with existing data


Below you have 2 generators. You will be tasked to load them to duckdb and answer some questions from the data

1. Load the first generator and calculate the sum of ages of all people. Make sure to only load it once.
2. Append the second generator to the same table as the first.
3. **After correctly appending the data, calculate the sum of all ages of people.**




In [2]:
def people_1():
    for i in range(1, 6):
        yield {"ID": i, "Name": f"Person_{i}", "Age": 25 + i, "City": "City_A"}

for person in people_1():
    print(person)


def people_2():
    for i in range(3, 9):
        yield {"ID": i, "Name": f"Person_{i}", "Age": 30 + i, "City": "City_B", "Occupation": f"Job_{i}"}


for person in people_2():
    print(person)


{'ID': 1, 'Name': 'Person_1', 'Age': 26, 'City': 'City_A'}
{'ID': 2, 'Name': 'Person_2', 'Age': 27, 'City': 'City_A'}
{'ID': 3, 'Name': 'Person_3', 'Age': 28, 'City': 'City_A'}
{'ID': 4, 'Name': 'Person_4', 'Age': 29, 'City': 'City_A'}
{'ID': 5, 'Name': 'Person_5', 'Age': 30, 'City': 'City_A'}
{'ID': 3, 'Name': 'Person_3', 'Age': 33, 'City': 'City_B', 'Occupation': 'Job_3'}
{'ID': 4, 'Name': 'Person_4', 'Age': 34, 'City': 'City_B', 'Occupation': 'Job_4'}
{'ID': 5, 'Name': 'Person_5', 'Age': 35, 'City': 'City_B', 'Occupation': 'Job_5'}
{'ID': 6, 'Name': 'Person_6', 'Age': 36, 'City': 'City_B', 'Occupation': 'Job_6'}
{'ID': 7, 'Name': 'Person_7', 'Age': 37, 'City': 'City_B', 'Occupation': 'Job_7'}
{'ID': 8, 'Name': 'Person_8', 'Age': 38, 'City': 'City_B', 'Occupation': 'Job_8'}


2. Append the second generator to the same table as the first.
3. **After correctly appending the data, calculate the sum of all ages of people.**

# 3. Merge a generator

Re-use the generators from Exercise 2.

A table's primary key needs to be created from the start, so load your data to a new table with primary key ID.

Load your first generator first, and then load the second one with merge. Since they have overlapping IDs, some of the records from the first load should be replaced by the ones from the second load.

After loading, you should have a total of 8 records, and ID 3 should have age 33.

Question: **Calculate the sum of ages of all the people loaded as described above.**


# Solution:

In [ ]:
#Install the dependencies
# I used poetry, so run `poetry install`

- **Question 1: What is the sum of the outputs of the generator for limit = 5?**

In [3]:
round(sum(list(square_root_generator(5))), 3)

8.382

- **Question 2: What is the 13th number yielded**

In [4]:
for i, sqrt_value in enumerate(square_root_generator(13)):
    print(i+1, round(sqrt_value, 3))

1 1.0
2 1.414
3 1.732
4 2.0
5 2.236
6 2.449
7 2.646
8 2.828
9 3.0
10 3.162
11 3.317
12 3.464
13 3.606


**Question 3 preparation**:

In [5]:
import dlt
import duckdb

1. Load the first generator and calculate the sum of ages of all people. Make sure to only load it once.

In [6]:
people_generator_pipeline = dlt.pipeline(
    destination='duckdb',
    dataset_name='people',
)

info = people_generator_pipeline.run(
    people_1(),
    table_name="people",
    write_disposition="replace",
)
print(info)

Pipeline dlt_ipykernel_launcher load step completed in 0.46 seconds
1 load package(s) were loaded to destination duckdb and into dataset people
The duckdb destination used duckdb:///C:\Users\staljaard\Source\dtc_de-zoomcamp_2024_homework\workshops\dlt\dlt_ipykernel_launcher.duckdb location to store data
Load package 1708012875.0567422 is LOADED and contains no failed jobs


In [7]:
conn = duckdb.connect(f"{people_generator_pipeline.pipeline_name}.duckdb")

def display_people(table_name):
    conn.sql(f"SET search_path = '{people_generator_pipeline.dataset_name}'")
    print('Loaded tables: ')
    display(conn.sql("show tables"))
    
    # and the data
    
    print("\n\n\n people table below:")
    
    display(conn.sql(f"SELECT * FROM people.{table_name}"))
    
    print("\n\n\n Total age:")
    print(conn.sql(f"SELECT SUM(age) from people.{table_name}"))

display_people("people")

Loaded tables: 


┌─────────────────────┐
│        name         │
│       varchar       │
├─────────────────────┤
│ _dlt_loads          │
│ _dlt_pipeline_state │
│ _dlt_version        │
│ people              │
└─────────────────────┘




 people table below:


┌───────┬──────────┬───────┬─────────┬────────────────────┬────────────────┐
│  id   │   name   │  age  │  city   │    _dlt_load_id    │    _dlt_id     │
│ int64 │ varchar  │ int64 │ varchar │      varchar       │    varchar     │
├───────┼──────────┼───────┼─────────┼────────────────────┼────────────────┤
│     1 │ Person_1 │    26 │ City_A  │ 1708012875.0567422 │ yDS0TXXEuS/NUA │
│     2 │ Person_2 │    27 │ City_A  │ 1708012875.0567422 │ wSjww77iMixTcA │
│     3 │ Person_3 │    28 │ City_A  │ 1708012875.0567422 │ E+pziL12T+bCcw │
│     4 │ Person_4 │    29 │ City_A  │ 1708012875.0567422 │ QDYW19zShuxr8w │
│     5 │ Person_5 │    30 │ City_A  │ 1708012875.0567422 │ 1RikhiTanLGfJg │
└───────┴──────────┴───────┴─────────┴────────────────────┴────────────────┘




 Total age:
┌──────────┐
│ sum(age) │
│  int128  │
├──────────┤
│      140 │
└──────────┘



**Question 3:**

2. Append the second generator to the same table as the first.
3. **After correctly appending the data, calculate the sum of all ages of people.**

In [8]:
people_generator_pipeline = dlt.pipeline(
    destination='duckdb',
    dataset_name='people',
)

info = people_generator_pipeline.run(
    people_2(),
    table_name="people",
    write_disposition="append",
)
print(info)

display_people("people")
conn.close()

Pipeline dlt_ipykernel_launcher load step completed in 0.30 seconds
1 load package(s) were loaded to destination duckdb and into dataset people
The duckdb destination used duckdb:///C:\Users\staljaard\Source\dtc_de-zoomcamp_2024_homework\workshops\dlt\dlt_ipykernel_launcher.duckdb location to store data
Load package 1708012876.6589866 is LOADED and contains no failed jobs
Loaded tables: 


┌─────────────────────┐
│        name         │
│       varchar       │
├─────────────────────┤
│ _dlt_loads          │
│ _dlt_pipeline_state │
│ _dlt_version        │
│ people              │
└─────────────────────┘




 people table below:


┌───────┬──────────┬───────┬─────────┬────────────────────┬────────────────┬────────────┐
│  id   │   name   │  age  │  city   │    _dlt_load_id    │    _dlt_id     │ occupation │
│ int64 │ varchar  │ int64 │ varchar │      varchar       │    varchar     │  varchar   │
├───────┼──────────┼───────┼─────────┼────────────────────┼────────────────┼────────────┤
│     1 │ Person_1 │    26 │ City_A  │ 1708012875.0567422 │ yDS0TXXEuS/NUA │ NULL       │
│     2 │ Person_2 │    27 │ City_A  │ 1708012875.0567422 │ wSjww77iMixTcA │ NULL       │
│     3 │ Person_3 │    28 │ City_A  │ 1708012875.0567422 │ E+pziL12T+bCcw │ NULL       │
│     4 │ Person_4 │    29 │ City_A  │ 1708012875.0567422 │ QDYW19zShuxr8w │ NULL       │
│     5 │ Person_5 │    30 │ City_A  │ 1708012875.0567422 │ 1RikhiTanLGfJg │ NULL       │
│     3 │ Person_3 │    33 │ City_B  │ 1708012876.6589866 │ hS2szRiEjsb2yA │ Job_3      │
│     4 │ Person_4 │    34 │ City_B  │ 1708012876.6589866 │ AM7gq5zXENa6BQ │ Job_4      │
│     5 │ 




 Total age:
┌──────────┐
│ sum(age) │
│  int128  │
├──────────┤
│      353 │
└──────────┘



**Question 4:**
**Calculate the sum of ages of all the people loaded in the merged table**

After loading, you should have a total of 8 records, and ID 3 should have age 33.

In [9]:
merged_table = "people_merged"

people_generator_merged_pipeline = dlt.pipeline(
    destination='duckdb',
    dataset_name="people",
)

info = people_generator_merged_pipeline.run(
    people_1(),
    table_name=merged_table,
    primary_key="id",
    write_disposition="replace",
)
print(info)
print('---')
info = people_generator_merged_pipeline.run(
    people_2(),
    table_name=merged_table,
    primary_key="id",
    write_disposition="merge",
)

Pipeline dlt_ipykernel_launcher load step completed in 0.44 seconds
1 load package(s) were loaded to destination duckdb and into dataset people
The duckdb destination used duckdb:///C:\Users\staljaard\Source\dtc_de-zoomcamp_2024_homework\workshops\dlt\dlt_ipykernel_launcher.duckdb location to store data
Load package 1708012879.9603498 is LOADED and contains no failed jobs
---


In [10]:
conn = duckdb.connect(f"{people_generator_merged_pipeline.pipeline_name}.duckdb")

print(conn.sql(f"SELECT COUNT(*) from people.{merged_table}").fetchall()[0][0] == 8)
print(conn.sql(f"SELECT age from people.{merged_table} WHERE id = 3").fetchall()[0][0] == 33)

display_people(merged_table)

conn.close()

True
True
Loaded tables: 


┌─────────────────────┐
│        name         │
│       varchar       │
├─────────────────────┤
│ _dlt_loads          │
│ _dlt_pipeline_state │
│ _dlt_version        │
│ people              │
│ people_merged       │
└─────────────────────┘




 people table below:


┌───────┬──────────┬───────┬─────────┬────────────────────┬────────────────┬────────────┐
│  id   │   name   │  age  │  city   │    _dlt_load_id    │    _dlt_id     │ occupation │
│ int64 │ varchar  │ int64 │ varchar │      varchar       │    varchar     │  varchar   │
├───────┼──────────┼───────┼─────────┼────────────────────┼────────────────┼────────────┤
│     1 │ Person_1 │    26 │ City_A  │ 1708012879.9603498 │ sqSmFtrmfajoeA │ NULL       │
│     2 │ Person_2 │    27 │ City_A  │ 1708012879.9603498 │ 6ElaFL9/V7kpyw │ NULL       │
│     8 │ Person_8 │    38 │ City_B  │ 1708012880.9328487 │ sAKJm+Qso3Tyng │ Job_8      │
│     5 │ Person_5 │    35 │ City_B  │ 1708012880.9328487 │ grYBKvRC+MysEA │ Job_5      │
│     4 │ Person_4 │    34 │ City_B  │ 1708012880.9328487 │ lWV1dS/fFxGbnA │ Job_4      │
│     3 │ Person_3 │    33 │ City_B  │ 1708012880.9328487 │ hdTeD9cG94BGGg │ Job_3      │
│     6 │ Person_6 │    36 │ City_B  │ 1708012880.9328487 │ /oTTke2l4yOtFg │ Job_6      │
│     7 │ 




 Total age:
┌──────────┐
│ sum(age) │
│  int128  │
├──────────┤
│      266 │
└──────────┘



In [11]:
conn.close()